In [ ]:
import cv2
import numpy as np
import scipy
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import pandas as pd
from PIL import Image

CORRECTION = 0.25

In [ ]:
bRunAWS = 1

In [ ]:
remove_top_pixels = 55
remove_bottom_pixels = 25
WIDTH,HEIGHT,CHANNELS = 64,64,3

# image resizing and cropping followed by normalization
def preprocess_image(collection_images):

    image = np.squeeze(collection_images[remove_top_pixels:-remove_bottom_pixels,:,:])
    image = cv2.resize(image,(WIDTH,HEIGHT))
    image = (image-128)/256
            
    return image

In [ ]:
# flip image horizontally and steering angle to simulate right turns
def flip_images(image, steering):

    image_mod = cv2.flip(image,1)
    steering_mod = steering*-1.0
    
    return (image_mod, steering_mod)

In [ ]:
# add random brightness to augment data and clip at 255
def brightness_adjust(image, steering):
    
    image_mod = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    random_bright = .25+np.random.uniform()
    image_mod[:,:,2] = image_mod[:,:,2]*random_bright
    image_mod[:,:,2][image_mod[:,:,2]>255] = 255
    image_mod = cv2.cvtColor(image_mod,cv2.COLOR_HSV2RGB)

    return (image_mod, steering)

My data augmentation step involved
1. Choosing either center, left or right image at random
2. Flipping the image horizontally at random and multipling steering value with -1.0
3. Multipling the image with random brightness to augment data
4. Finally I preprocess the image by removing top and bottom pixels to remove irrelevant pixels and reducing noise

In [ ]:
# Data generator for reading images
def data_generator(data, batch_size):
    
    ii = 0
    N = len(data)
        
    while True:
        
        start = ii*batch_size
        end = np.amin(((ii+1)*batch_size,N))    
        data_batches_files  = data[start:end]
        center_image_files = np.asarray(data_batches_files['center'])
        left_image_files = np.asarray(data_batches_files['left'])
        right_image_files = np.asarray(data_batches_files['right'])
        steering_arr = np.asarray(data_batches_files['steering'])

        X_batches = np.zeros((batch_size, WIDTH, HEIGHT, CHANNELS), dtype=np.float32)
        y_batches = np.zeros((batch_size,), dtype=np.float32)
        
        for kk in range(batch_size):
        # Choose either of the image randomly
            img_type = np.random.choice(['center','left','right'])
            if(img_type=='center'):
                image = plt.imread(path+center_image_files[kk])
                steering = steering_arr[kk]
            elif(img_type=='left'):    
                image = plt.imread(path+left_image_files[kk][1:])
                steering = steering_arr[kk]+CORRECTION
            elif(img_type=='right'):
                image = plt.imread(path+right_image_files[kk][1:])
                steering = steering_arr[kk]-CORRECTION
            
            # choose the flipped image based on random distribution   
            rand_num = np.random.random()
            if rand_num>0.5:
                image, steering = flip_images(image, steering)
            #add random brightness to image
            image, steering = brightness_adjust(image, steering)
            X_batches[kk,:,:,:] = preprocess_image(image)
            y_batches[kk] = steering
        #increment counter but reset when all images have been iterated    
        ii += 1
        if ii>=(N//batch_size):
            ii=0
    
        yield (X_batches, y_batches)


### Model Architecture -
The model that worked was inspired by the NVIDIA SDC model, for some reason it didn't work for me. It deviated into the first lake on right. The subsequent modifications was able to work. 

1. convolution2d_1 (Convolution2D)
2. elu_1 (ELU)
3. convolution2d_2 (Convolution2D)
4. elu_2 (ELU)
5. dropout_1 (Dropout)
6. maxpooling2d_1 (MaxPooling2D)
7. convolution2d_3 (Convolution2D)
8. elu_3 (ELU)
9. dropout_2 (Dropout)
10. convolution2d_4 (Convolution2D)
11. elu_4 (ELU)
12. dropout_3 (Dropout)
13. flatten_1 (Flatten)
14. dense_1 (Dense)
15. dropout_4 (Dropout)
16. dense_2 (Dense)
17. dense_3 (Dense)
18. dense_4 (Dense)

Total params: 8,517,473
Trainable params: 8,517,473
Non-trainable params: 0
________________________________

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Lambda, Convolution2D, Cropping2D, Dropout, ELU
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import Adam
#from keras.utils.visualize_util import plot

#nvidia SDC model as suggested in the exercise 
def nvidiaModel():
    model = Sequential()
    
    model.add(Convolution2D(24,5,5, input_shape=(WIDTH, HEIGHT, CHANNELS), subsample=(2,2), activation='relu'))
    model.add(Convolution2D(36,5,5, subsample=(2,2), activation='relu'))
    model.add(Convolution2D(48,5,5, subsample=(2,2), activation='relu'))
    model.add(Convolution2D(64,3,3, activation='relu'))
    model.add(Convolution2D(64,3,3, activation='relu'))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Dense(50))
    model.add(Dense(10))
    model.add(Dense(1))

    return model

#modified version of nvidia adding dropouts for regularization, etc.
def nvidiaModel_mod():

    model = Sequential()

    model.add(Convolution2D(24,8,8, input_shape=(WIDTH, HEIGHT, CHANNELS),subsample=(2,2)))
    model.add(ELU())
    model.add(Convolution2D(36,5,5, subsample=(2,2)))
    model.add(ELU())
    model.add(Convolution2D(48,3,3, subsample=(1,1)))
    model.add(ELU())
    model.add(Convolution2D(64,3,3))
    model.add(ELU())
    model.add(Convolution2D(64,3,3))
    model.add(ELU())
    model.add(Flatten())
    model.add(Dropout(.4))
    model.add(Dense(100))
    model.add(Dropout(.4))
    model.add(Dense(50))
    model.add(Dense(10))
    model.add(Dense(1))
    
    return model

# Custom model based on trial and error and inspired from nvidia model itself
def custom_model():
    model = Sequential()

    # layer 1 output shape is 32x32x32
    model.add(Convolution2D(16, 5, 5, input_shape=(WIDTH, HEIGHT, CHANNELS), subsample=(2, 2), border_mode="same"))
    model.add(ELU())

    # layer 2 output shape is 15x15x16
    model.add(Convolution2D(32, 3, 3, subsample=(1, 1), border_mode="valid"))
    model.add(ELU())
    model.add(Dropout(.4))
    model.add(MaxPooling2D((2, 2), border_mode='valid'))

    # layer 3 output shape is 12x12x16
    model.add(Convolution2D(64, 3, 3, subsample=(1, 1), border_mode="valid"))
    model.add(ELU())
    model.add(Dropout(.4))

    # layer 4 output shape is 12x12x16
    model.add(Convolution2D(64, 3, 3, subsample=(1, 1), border_mode="valid"))
    model.add(ELU())
    model.add(Dropout(.4))
    
    # Flatten the output
    model.add(Flatten())

    # layer 5
    model.add(Dense(1024))
    model.add(Dropout(.3))
    model.add(ELU())

    # layer 6
    model.add(Dense(512))
    model.add(Dropout(.2))
    model.add(ELU())

    # Finally a single output, since this is a regression problem
    model.add(Dense(1))

    return model

In [ ]:
# Balance data since majority are of 0 steering angle -> done to reduce bias of the model to drive straight
def balance_data(df_log, zero_pct=0.5):
    
    total_data_size = len(df_log)
    steering_arr = np.asarray(df_log['steering'])
    zero_idx = []  
    for ii in range(total_data_size):
        if np.absolute(steering_arr[[ii]]) <= 0.25:
            zero_idx.append(ii)

    nonzero_data_size = total_data_size - len(zero_idx)
    zero_data_size = int(zero_pct * nonzero_data_size / (1 - zero_pct))

    remove_idx = np.random.choice(zero_idx, total_data_size - zero_data_size - nonzero_data_size, replace=False)
    df_log = df_log.drop(df_log.index[remove_idx]) 
    
    return df_log

### Training strategy
The training strategy was to start from the NVIDIA model architecture and tweak all its hyperparameters, i.e. batch-size, data_size, epochs
This led to straight-forward improvements of adding regularization such as drop-out, tried L2 regularization but didnot help.
Used the intuition of gradually complex or deep but shrinking width, height kernels from earlier lecture to refine the Nvidia model. The model seems to work well however it deviates off the track after the bridge where one side of the road isnot present. To circumvent this issue tried adding the balancing data feature of dropping images with abs(steering_angle) less than 0.25. But this too didnot help. Would like to add more data/image but driving along the track, however could not even after 10-12 tries. Controlling the car in training mode is quite difficult hence had to rely completely on the images provided. 

In [ ]:
if __name__ == "__main__":

    BATCH_SIZE = 32
    
    #Select path according to where its run
    if bRunAWS==1:
        path = 'examples/data/'
    else:
        path = 'C:/Users/AVIK/Documents/Udacity Self Driving Cars/CarND-Behavioral-Cloning-P3-master/examples/data/'
    
    # read the .xls file
    df_log = pd.read_csv(path+"driving_log.csv")
    
    #drop near zero steering angle images to remove bias(added later but didn't help) 
    df_log = balance_data(df_log, zero_pct=0.5)
    
    center_image,left_image,right_image, steering = df_log['center'],df_log['left'],df_log['right'], df_log['steering']
    
    #shuffle data
    df_log = df_log.sample(frac=1).reset_index(drop=True)
    
    training_split = 0.8
    #split training and validation data after random shuffle or dataframe rows
    training_data_rows = df_log.loc[0:int(len(df_log)*training_split)]    
    validation_data_rows = df_log.loc[int(len(df_log)*training_split):]
    
    #adding the training/validation generator functions
    training_generator = data_generator(training_data_rows, batch_size=BATCH_SIZE)
    validation_data_generator = data_generator(validation_data_rows, batch_size=BATCH_SIZE)
    
    model = custom_model()
    model.summary()

    # Compile model with Adam optimizer with learning rate as hyper-parameter. Tried - 0.0001(works best), 0.0005, 0.0008
    model.compile(loss='mse', optimizer=Adam(lr=0.0001))
    
    #determines number of images/samples to iterate over during training
    samples_per_epoch = (len(training_data_rows)*30//BATCH_SIZE)*BATCH_SIZE 
    
    # fit.generator to train/validate the model
    history_object = model.fit_generator(training_generator, validation_data=validation_data_generator,
                        samples_per_epoch=samples_per_epoch, nb_epoch=3, nb_val_samples=len(validation_data_rows)*15)

    print("Saving model weights and configuration file.")

    model.save('model.h5')  
       

In [ ]:
### print the keys contained in the history object
print(history_object.history.keys())

### plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()